In [ ]:
import os

In [ ]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
import pandas as pd
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
from metrics import dice_loss, dice_coef, iou
from train import load_data, create_dir

In [ ]:
H = 256
W = 256

In [ ]:
def read_image(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)  # (H, W, 3)
    img = cv2.resize(img, (W, H))
    ori_img = img
    img = img / 255.0
    img = img.astype(np.float32)
    img = np.expand_dims(img, axis=0)
    return ori_img, img  # (1, 256, 256, 3)

In [ ]:
def read_mask(path):
    mask = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  # (H, W)
    mask = cv2.resize(mask, (W, H))
    ori_mask = mask
    mask = mask / 255.0
    mask = mask.astype(np.int32)  # (256, 256)
    return ori_mask, mask

In [ ]:
def save_results(ori_img, ori_mask, mask_pred, save_image_path):
    line = np.ones((H, 10, 3)) * 255

    ori_mask = np.expand_dims(ori_mask, axis=-1)  # (256, 256, 1)
    ori_mask = np.concatenate([ori_mask, ori_mask, ori_mask], axis=-1)  # (256, 256, 3)

    mask_pred = np.expand_dims(mask_pred, axis=-1)  # (256, 256, 1)
    mask_pred = np.concatenate([mask_pred, mask_pred, mask_pred], axis=-1)  # (256, 256, 3)

    cat_images = np.concatenate([ori_img, line, ori_mask, line, mask_pred * 255], axis=1)
    cv2.imwrite(save_image_path, cat_images)

In [ ]:
if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Folder for saving results """
    create_dir("results")

    """ Load the model """
    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
        model = tf.keras.models.load_model("files/split-0.4-1003-1003-imgs-masks-5-epoch/model.h5")

    """ Load the test data """
    dataset_path = "ISIC2018-Challenge/"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)

    SCORE = []
    for img, mask in tqdm(zip(test_x, test_y), total=len(test_x)):
        """ Exctracting the image name """
        name = img.split("\\")[-1]

        """ Read the image and mask """
        ori_img, img = read_image(img)
        ori_mask, mask = read_mask(mask)

        """ Predicting the mask """
        mask_pred = model.predict(img)[0] > 0.5
        mask_pred = np.squeeze(mask_pred, axis=-1)
        mask_pred = mask_pred.astype(np.int32)

        """ Saving the predicted mask """
        save_image_path = f"results/{name}"

        save_results(ori_img, ori_mask, mask_pred, save_image_path)

        """ Flatten the array """
        mask = mask.flatten()
        mask_pred = mask_pred.flatten()

        """ Calculating metrics values """
        acc_value = accuracy_score(mask, mask_pred)
        f1_value = f1_score(mask, mask_pred, labels=[0, 1], average="binary") # Quanto mais próximo de 0 pior
        jac_value = jaccard_score(mask, mask_pred, labels=[0, 1], average="binary")
        recall_value = recall_score(mask, mask_pred, labels=[0, 1], average="binary")
        precision_value = precision_score(mask, mask_pred, labels=[0, 1], average="binary")
        SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])

    """ mean metrics values """
    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f}")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")

    df = pd.DataFrame(SCORE, columns=["Image Name", "Acc", "F1", "Jaccard", "Recall", "Precision"])
    df.to_csv("files/split-0.4-1003-1003-imgs-masks-5-epoch/score.csv")